# Packed ensemble submission process

## Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import torch
import torch.nn.functional as F

from lips import get_root_path
from lips.dataset.scaler.standard_scaler import StandardScaler
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import download_data
from lips.augmented_simulators.torch_simulator import TorchSimulator
from lips.dataset.scaler.standard_scaler_iterative import StandardScalerIterative

#from my_custom_packed_ensemble import *
#from my_packed_cv import *
from my_augmented_simulator import *

## Generic Step (Load the required data) <a id='generic_step'></a>

In [ ]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = '../ml4physim_startingkit/Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [ ]:
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations", "benchmarks",
                                 "confAirfoil.ini")  #Configuration file related to the benchmark
SIM_CONFIG_PATH = r"config.ini"  #Configuration file re

Download the data

In [ ]:
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

Loading the dataset using the dedicated class used by LIPS platform offers a list of advantages:

1. Ease the importing of datasets
1. A set of functions to organize the `inputs` and `outputs` required by augmented simulators


In [ ]:
# Load the required benchmark datasets
def load_dataset():
    """
    Load the airfrans dataset as a benchmark object

    Returns
    -------
    benchmark : AirfRANSBenchmark
        The airfrans benchmark object
    """
    try:
        with open('benchmark.pkl', 'rb') as f:
            benchmark = pickle.load(f)
    except:
        benchmark = AirfRANSBenchmark(benchmark_path=DIRECTORY_NAME,
                                    config_path=BENCH_CONFIG_PATH,
                                    benchmark_name=BENCHMARK_NAME,
                                    log_path=LOG_PATH)
        benchmark.load(path=DIRECTORY_NAME)
        with open('benchmark.pkl', 'wb') as f:
            pickle.dump(benchmark, f)
    
    return benchmark

benchmark = load_dataset()

## Simulation

### Model creation

In [ ]:
"""
    Anthony : 0
    Anton   : 1
"""

partition = 0

In [ ]:
if partition == 0:  config_names = ["SMOOTH_G4", "BIG_MODEL"]
else:               config_names = ["DEFAULT", "SMOOTH_G1", "SMOOTH_G2"]

def simulate(config_names):
    """
    Creates a packed MLP model, trains it and evaluates it on the test dataset.

    Parameters
    ----------
    config_names : list
        List of the names of the configurations to be used for training the model.

    Returns
    -------
    int
        0 if we have successfully trained and evaluated the model.
    """
    
    for config_name in config_names:
        benchmark = load_dataset()

        chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
        no_norm_x=benchmark.train_dataset.get_no_normalization_axis_indices()
        scalerParams={"chunk_sizes":chunk_sizes,"no_norm_x":no_norm_x}

        # PackedMLP model definition 
        torch_sim = TorchSimulator(name="packed_mlp",
                           model=PackedMLP,
                           scaler=StandardScalerIterative,
                           scalerParams=scalerParams,
                           log_path=None,
                           device="cuda:0",
                           seed=42,
                           bench_config_path=BENCH_CONFIG_PATH,
                           bench_config_name="Benchmark1",
                           sim_config_path=SIM_CONFIG_PATH,
                           sim_config_name="DEFAULT",
                          )
        
        # model training 
        torch_sim.train(benchmark.train_dataset, 
                save_path=None, 
                epochs=3, 
                train_batch_size=128000,
                pin_memory=True, 
                non_blocking=True, 
                num_workers=6
                )
        
        # saving the model 
        torch_sim.save()

        # evaluating the model 
        torch_sim_metrics = benchmark.evaluate_simulator(augmented_simulator=torch_sim,
                                                  eval_batch_size=256000,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=".",
                                                  save_predictions=True
                                                 )
        
        return 0

In [ ]:
chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
no_norm_x=benchmark.train_dataset.get_no_normalization_axis_indices()
scalerParams={"chunk_sizes":chunk_sizes,"no_norm_x":no_norm_x}

torch_sim = TorchSimulator(name="packed_mlp",
                           model=PackedMLP,
                           scaler=StandardScalerIterative,
                           scalerParams=scalerParams,
                           log_path=None,
                           device="cuda:0",
                           seed=42,
                           bench_config_path=BENCH_CONFIG_PATH,
                           bench_config_name="Benchmark1",
                           sim_config_path=SIM_CONFIG_PATH,
                           sim_config_name="DEFAULT",
                          )

### Training

In [ ]:
torch_sim.train(benchmark.train_dataset, 
                save_path=None, 
                epochs=3, 
                train_batch_size=128000,
                pin_memory=True, 
                non_blocking=True, 
                num_workers=6
                )

### Metric evaluation

In [ ]:
torch_sim_metrics = benchmark.evaluate_simulator(augmented_simulator=torch_sim,
                                                  eval_batch_size=256000,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=".",
                                                  save_predictions=True
                                                 )

In [ ]:
torch_sim_metrics["test"]